In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

from scipy.optimize import minimize

In [2]:
# pd.set_option('display.max_rows', 20)

In [3]:
# import yfinance as yf
# import pandas as pd
# import numpy as np
# from scipy import optimize
# 
# import os
# cwd = os.getcwd()
# print(cwd)
# os.chdir(r'C:\Users\mitch\Documents\QCF\Finance and Investments\StockTrack-Trading-Simulation')

# Reusable functions

In [4]:
def calculate_return(df):
    if df.empty:
        return None
    # Get the closing price at the start of the year
    start_price = df['Close'].iloc[0]

    # Get the closing price at the end of the year
    end_price = df['Close'].iloc[-1]

    # Calculate the one-year return
    return (end_price - start_price) / start_price
# 
# 
# def MaximizeSharpeRatioOptmzn(MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
#     # define maximization of Sharpe Ratio using principle of duality
#     def f(x, MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
#         funcDenomr = np.sqrt(np.matmul(np.matmul(x, CovarReturns), x.T))
#         funcNumer = np.matmul(np.array(MeanReturns), x.T) - RiskFreeRate
#         func = -(funcNumer / funcDenomr)
#         return func
# 
#     #define equality constraint representing fully invested portfolio
#     def constraintEq(x):
#         A = np.ones(x.shape)
#         b = 1
#         constraintVal = np.matmul(A, x.T) - b
#         return constraintVal
# 
#     #define bounds and other parameters
#     xinit = np.repeat(0.33, PortfolioSize)
#     cons = ({'type': 'eq', 'fun': constraintEq})
#     lb = 0
#     ub = 1
#     bnds = tuple([(lb, ub) for x in xinit])
# 
#     #invoke minimize solver
#     opt = minimize(f, x0=xinit, args=(MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize), method='SLSQP',
#                    bounds=bnds, constraints=cons, tol=10 ** -3)
# 
#     return opt


In [5]:
largest_market_cap = pd.read_excel('./Top Companies by Market Cap.xlsx', sheet_name='Sheet1')
largest_market_cap

,Symbol,Company Name,Industry,Market Cap
0,AAPL,Apple Inc.,Consumer Electronics,"2,811.03B"
1,MSFT,Microsoft Corporation,Software - Infrastructure,"2,469.80B"
2,GOOGL,Alphabet Inc.,Internet Content & Information,"1,779.01B"
3,GOOG,Alphabet Inc.,Internet Content & Information,"1,778.79B"
4,AMZN,"Amazon.com, Inc.",Internet Retail,"1,360.20B"
...,...,...,...,...
495,RBLX,Roblox Corporation,Electronic Gaming & Multimedia,19.59B
496,WBA,"Walgreens Boots Alliance, Inc.",Pharmaceutical Retailers,19.51B
497,TW,Tradeweb Markets Inc.,Capital Markets,19.51B
498,ZM,"Zoom Video Communications, Inc.",Software - Application,19.51B


In [6]:
largest_market_cap = largest_market_cap[~largest_market_cap['Symbol'].str.contains('\.')].reset_index(drop=True)
top = largest_market_cap.head(100)
top_symbols = top['Symbol']

In [7]:
# ticker = yf.Ticker('AAPL')
# stock_info = ticker.info
# print(stock_info.get('beta'))

In [8]:
one_year_data = {}
three_year_data = {}
five_year_data = {}
for symbol in top_symbols:
    ticker = yf.Ticker(symbol)
    one_year_data[symbol] = ticker.history(period="1y")
    three_year_data[symbol] = ticker.history(period="3y")
    five_year_data[symbol] = ticker.history(period="5y")

In [9]:
one_year_return_data = {}
three_year_return_data = {}
five_year_return_data = {}
for key, df in one_year_data.items():
    one_year_return_data[key] = calculate_return(one_year_data[key])
    three_year_return_data[key] = calculate_return(three_year_data[key])
    five_year_return_data[key] = calculate_return(five_year_data[key])

one_year_return_data

{'AAPL': 0.2937794232613147,
 'MSFT': 0.5507082393316682,
 'GOOGL': 0.43101459057883085,
 'GOOG': 0.44458932394314615,
 'AMZN': 0.5523469630165798,
 'NVDA': 2.2522331512362626,
 'META': 2.088248607423694,
 'TSLA': 0.43567466264686144,
 'LLY': 0.6596676265139012,
 'V': 0.23156711759453585,
 'UNH': 0.06149647978507185,
 'TSM': 0.26079582167227,
 'NVO': 0.8667653377406781,
 'WMT': 0.05154871113872603,
 'JPM': 0.18530533988270698,
 'XOM': -0.02741073016241829,
 'JNJ': -0.11744092703072755,
 'MA': 0.2186543003128291,
 'AVGO': 0.950813791304629,
 'PG': 0.06242337035830541,
 'CVX': -0.1744833620621768,
 'ORCL': 0.47846347347968216,
 'HD': -0.009102905690481219,
 'ABBV': -0.08204457731217486,
 'MRK': -0.004165440227472742,
 'COST': 0.1249401911626744,
 'ADBE': 0.8912253203353829,
 'TM': 0.29558235742274164,
 'ASML': 0.18594687869411397,
 'KO': -0.03873827978148943,
 'PEP': -0.06814586886661313,
 'BABA': 0.026647641279417624,
 'SHEL': 0.2627302990968603,
 'CSCO': 0.03671300654910915,
 'BAC': -0

In [10]:
def get_deciles(data):
    # Remove None values and sort the data
    filtered_data = {k: v for k, v in data.items() if v is not None}
    sorted_data = sorted(filtered_data.items(), key=lambda x: x[1])

    # Calculate the index for decile
    n = len(sorted_data)
    decile_index = n // 10

    # Get the top and bottom decile
    bottom_decile = dict(sorted_data[:decile_index])
    top_decile = dict(sorted_data[-decile_index:])

    return top_decile, bottom_decile


# Apply the function to each dataset
top_decile_one_year, bottom_decile_one_year = get_deciles(one_year_return_data)
top_decile_three_year, bottom_decile_three_year = get_deciles(three_year_return_data)
top_decile_five_year, bottom_decile_five_year = get_deciles(five_year_return_data)

# Find overlapping keys in the top deciles
overlapping_top_keys = set(top_decile_one_year.keys()) & set(top_decile_three_year.keys()) & set(
    top_decile_five_year.keys())

# Find overlapping keys in the bottom deciles
overlapping_bottom_keys = set(bottom_decile_one_year.keys()) & set(bottom_decile_three_year.keys())

# Print results
print("Top Decile Overlaps:", overlapping_top_keys)
print("Bottom Decile Overlaps:", overlapping_bottom_keys)


Top Decile Overlaps: {'AVGO', 'NVDA', 'LLY', 'NVO'}
Bottom Decile Overlaps: {'HDB', 'NEE', 'BMY'}


# Choosing top-50 and bottom-50 companies

In [11]:
'''''

target_long = pd.DataFrame(['ASML', 'GOOG', 'ACN', 'AAPL', 'SONY',
                            'INTU', 'V', 'MA', 'BX', 'TTE'],
                           columns=['Ticker'])
# TODO
target_short = pd.DataFrame([],
                            columns=['Ticker']
                            )
                            
                            '''''

"''\n\ntarget_long = pd.DataFrame(['ASML', 'GOOG', 'ACN', 'AAPL', 'SONY',\n                            'INTU', 'V', 'MA', 'BX', 'TTE'],\n                           columns=['Ticker'])\n# TODO\ntarget_short = pd.DataFrame([],\n                            columns=['Ticker']\n                            )\n                            \n                            "

In [12]:
# Get all unique keys in the top deciles
all_top_keys = set(top_decile_one_year.keys()) | set(top_decile_three_year.keys()) | set(top_decile_five_year.keys())

# Get all unique keys in the bottom deciles
all_bottom_keys = set(bottom_decile_one_year.keys()) | set(bottom_decile_three_year.keys()) | set(
    bottom_decile_five_year.keys())

all_top_keys

{'AAPL',
 'ADBE',
 'AMAT',
 'AMD',
 'ASML',
 'AVGO',
 'BP',
 'BX',
 'COP',
 'GE',
 'LLY',
 'META',
 'NFLX',
 'NOW',
 'NVDA',
 'NVO',
 'ORCL',
 'PDD',
 'SHEL',
 'TSLA',
 'XOM'}

In [13]:
all_bottom_keys

{'BA',
 'BABA',
 'BAC',
 'BMY',
 'BUD',
 'CRM',
 'CVX',
 'DIS',
 'HDB',
 'HON',
 'INTC',
 'JNJ',
 'NEE',
 'NKE',
 'PDD',
 'PFE',
 'RTX',
 'UL',
 'UPS',
 'VZ',
 'WFC'}

In [14]:
target_long = pd.DataFrame(list(all_top_keys),
                           columns=['Ticker'])
# TODO
target_short = pd.DataFrame(list(all_bottom_keys),
                            columns=['Ticker'])


# Getting 2-month history

In [15]:
def get_history(target_df: pd.DataFrame, long=True) -> pd.DataFrame:
    history = pd.DataFrame(columns=list(target_long['Ticker']))
    for ticker in target_df['Ticker']:
        # TODO think about the appropriate price column to use
        history[ticker] = yf.Ticker(ticker).history(interval='5m')['High' if long else 'Low']
    return history

In [16]:
history_long = get_history(target_long)
history_long.info()
history_long.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1689 entries, 2023-10-23 09:30:00-04:00 to 2023-11-21 13:40:00-05:00
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SHEL    1689 non-null   float64
 1   NOW     1689 non-null   float64
 2   AAPL    1689 non-null   float64
 3   ASML    1687 non-null   float64
 4   XOM     1689 non-null   float64
 5   LLY     1689 non-null   float64
 6   ORCL    1689 non-null   float64
 7   COP     1689 non-null   float64
 8   BX      1689 non-null   float64
 9   META    1689 non-null   float64
 10  AMD     1689 non-null   float64
 11  NFLX    1689 non-null   float64
 12  PDD     1689 non-null   float64
 13  GE      1689 non-null   float64
 14  ADBE    1689 non-null   float64
 15  AVGO    1689 non-null   float64
 16  BP      1689 non-null   float64
 17  AMAT    1689 non-null   float64
 18  TSLA    1689 non-null   float64
 19  NVDA    1689 non-null   float64
 20  NVO     1689 non-null   fl

,SHEL,NOW,AAPL,ASML,XOM,LLY,ORCL,COP,BX,META,...,NFLX,PDD,GE,ADBE,AVGO,BP,AMAT,TSLA,NVDA,NVO
Datetime,,,,,,,,,,,,,,,,,,,,,
2023-10-23 09:30:00-04:00,67.690002,540.960022,171.679993,579.190002,110.959999,586.059998,102.919998,123.160004,94.300003,310.649994,...,407.109985,103.190002,107.440002,537.929993,858.765198,39.200001,133.985001,210.479996,414.260010,97.139999
2023-10-23 09:35:00-04:00,67.470001,536.000000,171.054993,577.005005,110.769997,580.919983,103.019997,122.400002,93.230003,310.200012,...,407.509888,104.129898,107.989998,534.340027,847.530029,39.009998,133.059998,206.729996,411.746887,96.720001
2023-10-23 09:40:00-04:00,67.300003,532.080017,170.570007,575.349976,110.669998,580.359985,102.910004,122.720001,93.059998,309.380005,...,404.700012,103.889999,107.959999,533.044983,845.780029,38.880501,132.839905,205.250000,412.484100,96.599998
2023-10-23 09:45:00-04:00,67.209999,537.859985,171.100006,580.280029,109.875000,579.250000,102.919998,122.180000,93.635002,309.970001,...,406.480011,102.772202,107.680000,535.726318,851.630005,38.750000,133.890594,205.500000,416.299896,96.800003
2023-10-23 09:50:00-04:00,67.260002,538.429993,171.350006,581.140015,110.269997,578.989990,102.699997,122.459999,93.720001,310.649994,...,405.763611,103.000000,107.769997,537.789978,852.200012,38.790001,134.100006,206.990005,418.119995,96.860001


In [17]:
history_short = get_history(target_short)
history_short.dropna(axis=1, inplace=True)
history_short.info()
history_short.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1689 entries, 2023-10-23 09:30:00-04:00 to 2023-11-21 13:40:00-05:00
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PDD     1689 non-null   float64
 1   HON     1689 non-null   float64
 2   DIS     1689 non-null   float64
 3   WFC     1689 non-null   float64
 4   BABA    1689 non-null   float64
 5   INTC    1689 non-null   float64
 6   NEE     1689 non-null   float64
 7   CVX     1689 non-null   float64
 8   CRM     1689 non-null   float64
 9   PFE     1689 non-null   float64
 10  NKE     1689 non-null   float64
 11  UL      1689 non-null   float64
 12  VZ      1689 non-null   float64
 13  UPS     1689 non-null   float64
 14  BAC     1689 non-null   float64
 15  BA      1689 non-null   float64
 16  RTX     1689 non-null   float64
 17  BUD     1689 non-null   float64
 18  JNJ     1689 non-null   float64
 19  HDB     1689 non-null   float64
 20  BMY     1689 non-null   fl

,PDD,HON,DIS,WFC,BABA,INTC,NEE,CVX,CRM,PFE,...,UL,VZ,UPS,BAC,BA,RTX,BUD,JNJ,HDB,BMY
Datetime,,,,,,,,,,,,,,,,,,,,,
2023-10-23 09:30:00-04:00,103.190002,181.869904,82.320000,40.365002,80.050003,34.764999,51.560001,164.429993,200.446594,30.870001,...,48.430000,31.490000,151.460007,26.150000,180.119995,72.860001,52.660000,152.869995,57.122101,56.430000
2023-10-23 09:35:00-04:00,104.129898,181.800003,81.809998,40.380001,80.004997,34.750000,51.540001,162.539993,199.580002,31.115000,...,48.419998,31.450001,150.830002,26.150000,180.500000,72.860001,52.549999,153.500000,56.970001,56.570000
2023-10-23 09:40:00-04:00,103.889999,181.639999,81.510002,40.139999,79.760002,34.495899,51.480000,162.970001,198.990005,31.160000,...,48.391499,31.440001,150.889999,26.040001,179.509995,72.912003,52.528801,153.399994,56.849998,56.580002
2023-10-23 09:45:00-04:00,102.772202,182.169998,81.510002,40.165001,79.730003,34.630001,51.560001,162.860001,199.800003,31.389999,...,48.450001,31.485001,151.240005,26.000000,180.080002,72.970001,52.619999,153.294998,56.970001,56.650002
2023-10-23 09:50:00-04:00,103.000000,182.479996,81.799698,40.189999,79.769997,34.689999,51.660000,163.029999,200.550003,31.260000,...,48.430000,31.549999,151.580002,26.010000,180.389999,73.190002,52.735001,153.315002,57.046101,56.684700


# Getting 2-month compound returns

In [18]:
def get_returns(history_df: pd.DataFrame) -> np.ndarray:
    n = len(history_df.columns)
    returns = np.zeros(n, dtype=float)
    for i in range(n):
        # TODO think about the appropriate return calculation
        helper = history_df.iloc[:, i] / history_df.iloc[:, i].shift(1)
        helper = helper.iloc[1:]
        returns[i] = np.prod(helper) - 1
    return returns

In [19]:
returns_long = get_returns(history_long)
print(f'Top companies\' returns: {returns_long}')

Top companies' returns: [-0.01913142  0.2330948   0.10933134  0.17628676 -0.05903024  0.01296896
  0.12976294 -0.0667222   0.12576883  0.09267638  0.16862127  0.17113315
  0.13779822  0.11639052  0.13054419  0.14541412 -0.09005108  0.10751203
  0.14528226  0.20272288  0.06238416]


In [20]:
returns_short = get_returns(history_short)
print(f'Bottom companies\' returns: {returns_short}')

Bottom companies' returns: [ 0.13779822  0.05058618  0.14856661  0.05475529 -0.01305441  0.25269382
  0.12563804 -0.12424736  0.11979955 -0.02184646  0.03908158 -0.01796405
  0.18228008 -0.01010177  0.1357553   0.21230295  0.07672243  0.17650972
 -0.01396608  0.03051185 -0.13775471]


# Getting weights

In [21]:
def sharpe_long(weights):
    return -np.dot(returns_long, weights) / np.sqrt(weights @ history_long.cov().to_numpy() @ weights)


def sharpe_short(weights):
    return -np.dot(returns_short, weights) / np.sqrt(weights @ history_short.cov().to_numpy() @ weights)

In [22]:
'''''
def get_weights(history_df: pd.DataFrame, long=True) -> np.ndarray:
    n = len(history_df.columns)
    weights = np.ones(n)

    bounds = [(0.0, None)] * n if long else [(None, 0.0)] * n
    constraints = ({'type': 'eq', 'fun': lambda weights: weights.sum() - 1.0})
    optimal_weights = minimize(sharpe_long if long else sharpe_short,
                               weights, 
                               method='SLSQP',
                               constraints=constraints,
                               bounds=bounds
                               ).x

    return optimal_weights
    '''''

"''\ndef get_weights(history_df: pd.DataFrame, long=True) -> np.ndarray:\n    n = len(history_df.columns)\n    weights = np.ones(n)\n\n    bounds = [(0.0, None)] * n if long else [(None, 0.0)] * n\n    constraints = ({'type': 'eq', 'fun': lambda weights: weights.sum() - 1.0})\n    optimal_weights = minimize(sharpe_long if long else sharpe_short,\n                               weights, \n                               method='SLSQP',\n                               constraints=constraints,\n                               bounds=bounds\n                               ).x\n\n    return optimal_weights\n    "

In [23]:
def get_weights(history_df: pd.DataFrame, long=True) -> dict:
    n = len(history_df.columns)
    weights = np.ones(n) if long else np.full(n, -1)

    bounds = [(0.0, 0.10)] * n if long else [(-0.1, 0.0)] * n
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0}) if long else \
        ({'type': 'eq', 'fun': lambda w: np.sum(w) + 1.0})
    optimal_weights = minimize(sharpe_long if long else sharpe_short,
                               weights,
                               method='SLSQP',
                               constraints=constraints,
                               bounds=bounds
                               ).x

    # Create a dictionary with column names as keys and weights as values
    weights_dict = dict(zip(history_df.columns, optimal_weights))
    return weights_dict

In [24]:
weights_long = get_weights(history_long, True)
weights_long_vw = pd.DataFrame(list(weights_long.items()), columns=['Stock', 'Weight'])

weights_long_vw['Weight'] = weights_long_vw['Weight'].apply(lambda x: f"{x * 100:.2f}%")

#print(f'Optimal weights for top companies: {weights_long}')

#print(f'Optimal Sharpe ratio: {-sharpe_long(weights_long)}')

#history_long.columns

print(weights_long_vw)



   Stock  Weight
0   SHEL  10.00%
1    NOW   0.00%
2   AAPL  10.00%
3   ASML   0.00%
4    XOM  10.00%
5    LLY   0.00%
6   ORCL  10.00%
7    COP  10.00%
8     BX  10.00%
9   META   0.00%
10   AMD  10.00%
11  NFLX   0.00%
12   PDD  10.00%
13    GE  10.00%
14  ADBE   0.00%
15  AVGO   0.00%
16    BP   0.00%
17  AMAT   0.00%
18  TSLA   0.00%
19  NVDA   0.00%
20   NVO  10.00%


In [25]:
weights_short = get_weights(history_short, False)
weights_short_vw = pd.DataFrame(list(weights_short.items()), columns=['Stock', 'Weight'])
weights_short_vw['Weight'] = weights_short_vw['Weight'].apply(lambda x: f"{x * 100:.2f}%")

#print(f'Optimal weights for top companies: {weights_long}')

#print(f'Optimal Sharpe ratio: {-sharpe_long(weights_long)}')

#history_long.columns

print(weights_short_vw)

# print(f'Optimal weights for bottom companies: {weights_short}')
# print(f'Optimal Sharpe ratio: {-sharpe_short(np.array(list(weights_short.values())))}')

   Stock   Weight
0    PDD    0.00%
1    HON   -0.00%
2    DIS    0.00%
3    WFC  -10.00%
4   BABA  -10.00%
5   INTC    0.00%
6    NEE    0.00%
7    CVX  -10.00%
8    CRM   -0.00%
9    PFE  -10.00%
10   NKE  -10.00%
11    UL  -10.00%
12    VZ    0.00%
13   UPS  -10.00%
14   BAC    0.00%
15    BA    0.00%
16   RTX   -0.00%
17   BUD    0.00%
18   JNJ  -10.00%
19   HDB  -10.00%
20   BMY  -10.00%
